In [3]:
import pandas as pd 
import numpy as np

In [4]:
df = pd.read_csv("dados_ajustados.csv", delimiter=';', decimal=',', index_col='Data')

In [20]:
class CovarianceEstimator:
    
    def __init__(self, returns):
        self.returns = returns.dropna()
    
    def honey_shrinkage(self):
        X = self.returns.values
        s = self.returns.cov().values
        n = s.shape[0]
        F = np.zeros((n,n))
        
        
        #### Estimando F
        var = self.returns.var().values
        corr = self.returns.corr().values
        n = corr.shape[0]
        r = (np.sum(corr) - corr.shape[0])/(n**2-n)
        
        
        for i in range(n):
            for j in range(n):
                if i == j:
                    F[i][i] = var[i]
                else:
                    F[i][j] = r*np.sqrt(var[i]*var[j])
          
        
        returns_media = self.returns-self.returns.mean()
        pi = 0
        for i in range(n):
            for j in range(n):
                a =1/returns_media.shape[0] *np.sum(((returns_media.iloc[:, i]*returns_media.iloc[:, 1])**2).values)
                pi = pi+a

        
        print(print(pi))
        print(self.returns.corr())
    
        return s

In [25]:
class CovarianceEstimator:
    
    def __init__(self, returns):
        self.returns = returns.dropna()
        self.S = returns.cov().values
    
    def honey_shrinkage(self):
        """
        Pegamos da biblioteca pyPortfolioopt.
        """
        
        
        X = np.nan_to_num(self.returns.values)
        t, n = np.shape(X)

        S = self.S  # sample cov matrix

        # Constant correlation target
        var = np.diag(S).reshape(-1, 1)
        std = np.sqrt(var)
        _var = np.tile(var, (n,))
        _std = np.tile(std, (n,))
        r_bar = (np.sum(S / (_std * _std.T)) - n) / (n * (n - 1))
        F = r_bar * (_std * _std.T)
        F[np.eye(n) == 1] = var.reshape(-1)

        # Estimate pi
        Xm = X - X.mean(axis=0)
        y = Xm ** 2
        pi_mat = np.dot(y.T, y) / t - 2 * np.dot(Xm.T, Xm) * S / t + S ** 2
        pi_hat = np.sum(pi_mat)

        # Theta matrix, expanded term by term
        term1 = np.dot((Xm ** 3).T, Xm) / t
        help_ = np.dot(Xm.T, Xm) / t
        help_diag = np.diag(help_)
        term2 = np.tile(help_diag, (n, 1)).T * S
        term3 = help_ * _var
        term4 = _var * S
        theta_mat = term1 - term2 - term3 + term4
        theta_mat[np.eye(n) == 1] = np.zeros(n)
        rho_hat = sum(np.diag(pi_mat)) + r_bar * np.sum(
            np.dot((1 / std), std.T) * theta_mat
        )

        # Estimate gamma
        gamma_hat = np.linalg.norm(S - F, "fro") ** 2

        # Compute shrinkage constant
        kappa_hat = (pi_hat - rho_hat) / gamma_hat
        delta = max(0.0, min(1.0, kappa_hat / t))

        # Compute shrunk covariance matrix
        shrunk_cov = delta * F + (1 - delta) * S
        return shrunk_cov, delta

In [23]:
a = CovarianceEstimator(df.pct_change())
a.honey_shrinkage()

(array([[ 2.11616664e-04,  2.77392650e-05, -8.85529330e-06,
         -4.50948667e-06],
        [ 2.77392650e-05,  2.21610254e-04,  2.42372255e-05,
          1.23196785e-05],
        [-8.85529330e-06,  2.42372255e-05,  1.47806359e-05,
          5.61930131e-06],
        [-4.50948667e-06,  1.23196785e-05,  5.61930131e-06,
          3.98535755e-06]]),
 0.21796513871096473)

In [24]:
df.pct_change().dropna().cov()

,SPXI,IBOV,IMAB,IRFM
SPXI,0.000212,0.000022,-0.000015,-0.000008
IBOV,0.000022,0.000222,0.000027,0.000014
IMAB,-0.000015,0.000027,0.000015,0.000007
IRFM,-0.000008,0.000014,0.000007,0.000004


In [17]:
(df.iloc[:,1])**2

Data
15/09/2020    1.005967e+10
16/09/2020    9.935241e+09
17/09/2020    1.001958e+10
18/09/2020    9.660867e+09
21/09/2020    9.407200e+09
                  ...     
29/03/2021    1.332148e+10
30/03/2021    1.365385e+10
31/03/2021    1.360342e+10
01/04/2021    1.328333e+10
05/04/2021    1.381058e+10
Name: IBOV, Length: 135, dtype: float64

In [9]:
r = df.pct_change().dropna()
r = r - r.mean()

In [11]:
df.shape

(135, 4)